# Programming Assignment: Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Нас будет интересовать файл `checkins.dat`. Открыв его, увидим следующую структуру:


    id | user_id | venue_id | latitude | longitude | created_at

    ---------+---------+----------+-------------------+-------------------+---------------------

    984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

    984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

    984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

    984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

    ...

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки не содержащие координат - они неинформативны для нас:

    id,user_id,venue_id,latitude,longitude,created_at

    984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

    984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

    984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

    ...

С помощью pandas построим `DataFrame` и убедимся, что все 396634 строки с координатами считаны успешно.

In [1]:
import pandas as pd
from sklearn.cluster import MeanShift

In [2]:
df = pd.read_csv('checkins.dat', sep='|', header=0, skipinitialspace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.rename(columns=lambda x: x.strip(), inplace=True)
print(df.shape)
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(396634, 6)


id    user_id  venue_id   latitude   longitude           created_at
0  984222     15824.0    5222.0  38.895112  -77.036366  2012-04-21 17:43:47
1  984234     44652.0    5222.0  33.800745  -84.410520  2012-04-21 17:43:43
2  984291    105054.0    5222.0  45.523452 -122.676207  2012-04-21 17:39:22
3  984318   2146539.0    5222.0  40.764462 -111.904565  2012-04-21 17:35:46
4  984232     93870.0  380645.0  33.448377 -112.074037  2012-04-21 17:38:18

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом `MeanShift`, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [`sklearn user guide`](http://scikit-learn.org/stable/modules/clustering.html#mean-shift), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте `MeanShift`, указав `bandwidth=0.1`, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

**Примечание**:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт [`mapcustomizer.com`](https://www.mapcustomizer.com/) имеет функцию `Bulk Entry`, куда можно вставить центры полученных кластеров в формате:

    38.8951118,-77.0363658

    33.800745,-84.41052

    45.5234515,-122.6762071

    ...

In [3]:
X = df.iloc[:100000,3:5]
print(X.shape)
X.head()

(100000, 2)


latitude   longitude
0  38.895112  -77.036366
1  33.800745  -84.410520
2  45.523452 -122.676207
3  40.764462 -111.904565
4  33.448377 -112.074037

In [4]:
clustering = MeanShift(bandwidth=0.1).fit(X)

In [5]:
X['cluster'] = clustering.labels_
X.head()

latitude   longitude  cluster
0  38.895112  -77.036366        5
1  33.800745  -84.410520        7
2  45.523452 -122.676207       30
3  40.764462 -111.904565       65
4  33.448377 -112.074037        1

In [6]:
cluster_size = pd.DataFrame(X.pivot_table(values = 'latitude', index = 'cluster', aggfunc = 'count'))
cluster_size.columns = ['clust_size']
cluster_size.head()

clust_size
cluster            
0             12506
1              4692
2              3994
3              3363
4              3526

In [7]:
cluster_centers_df = pd.DataFrame(clustering.cluster_centers_)
cluster_centers_df.columns = ['cent_latitude', 'cent_longitude']
cluster_centers_df.head()

cent_latitude  cent_longitude
0      40.717716      -73.991835
1      33.449438     -112.002140
2      33.446380     -111.901888
3      41.878244      -87.629843
4      37.688682     -122.409330

In [8]:
cluster_df = cluster_centers_df.join(cluster_size)
cluster_df.to_csv('clusters.csv', index = None)
cluster_df = cluster_df[cluster_df.clust_size > 15]
cluster_df.head(10)

cent_latitude  cent_longitude  clust_size
0      40.717716      -73.991835       12506
1      33.449438     -112.002140        4692
2      33.446380     -111.901888        3994
3      41.878244      -87.629843        3363
4      37.688682     -122.409330        3526
5      38.886165      -77.048783        2409
6      33.357345     -111.822654        2297
7      33.766636      -84.393289        1601
8      42.363219      -71.073688        1526
9      47.606245     -122.332044        1378

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

    33.751277, -118.188740 (_Los Angeles_)

    25.867736, -80.324116 (_Miami_)

    51.503016, -0.075479 (_London_)

    52.378894, 4.885084 (_Amsterdam_)

    39.366487, 117.036146 (_Beijing_)

    -33.868457, 151.205134 (_Sydney_)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании - широта и долгота этого центра, записанные через пробел.

In [9]:
def get_distance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5

def get_min_distance_to_office(lat, lon):
    
    office_coordinates = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
    ]
    
    min_dist = None
    for (of_lat, of_lon) in office_coordinates:
        dist = get_distance(lat, lon, of_lat, of_lon)
        if (min_dist is None) or (dist < min_dist):
            min_dist = dist
    return min_dist

In [10]:
cluster_df['min_distance'] = list(map(get_min_distance_to_office, cluster_df.cent_latitude, cluster_df.cent_longitude))

near_centers = cluster_df.sort_values('min_distance')[:20]
near_centers

cent_latitude  cent_longitude  clust_size  min_distance
420     -33.860630      151.204776          28      0.007835
370      52.372964        4.892317          31      0.009353
419      25.845672      -80.318891          28      0.022674
58       51.502991       -0.125537         254      0.050058
51       33.809878     -118.148924         281      0.070848
29       25.785812      -80.217938         564      0.134109
167      25.705350      -80.283429          80      0.167406
92       26.010098      -80.199991         138      0.188876
87       33.888325     -118.048928         100      0.195779
42       33.872986     -118.362091         384      0.211811
291      33.972575     -118.168371          37      0.222233
320      26.138844      -80.334347          38      0.271301
119      33.983936     -118.007405          74      0.294979
55       26.120863      -80.158907         246      0.302270
27       33.817306     -117.891249         577      0.304731
11       34.060398     -118.248709        1081      0.314884
32       33.674303     -117.858789         449      0.338810
159      26.200585      -80.250716          42      0.340846
17       34.035487     -118.438998         645      0.378688
47       34.131460     -118.118012         273      0.386706

In [11]:
ans = str(near_centers.iloc[0][:2].values)
print('Координаты центра, наименее удаленного от ближайшего к нему офиса:', ans)

Координаты центра, наименее удаленного от ближайшего к нему офиса: [-33.86063043 151.20477593]


In [12]:
def write_answer(ans, n):
    with open("ans{}.txt".format(n), "w") as fout:
        fout.write(str(ans))
        
write_answer(ans[1:-1], 1)